In [314]:
from yfpy.query import YahooFantasySportsQuery
from dotenv import find_dotenv, load_dotenv
import os
import pandas as pd
import numpy as np

In [315]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir='auth/',
    league_id='1411473',
)

In [316]:
cereal = yahoo_query.get_league_standings().serialized()

2022-11-01 19:21:31.254 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.


In [317]:
l_standings = dict()
for team in cereal['teams']:
    team_info = team['team'].serialized()
    name = team_info['name'].decode()
    standings = team_info['team_standings']
    wins = standings['outcome_totals']['wins']
    losses = standings['outcome_totals']['losses']
    l_standings[name] = {'wins': wins, 'losses': losses}

In [318]:
# standings_pd = pd.DataFrame(l_standings).head()
# standings_pd['name'] = standings_pd['name'].str.decode("utf-8")

In [319]:
matchups = []
for week in range(9,14):
    for matchup in yahoo_query.get_league_matchups_by_week(week):
        matchup_teams = matchup['matchup'].serialized()['teams']
        matchups.append(
            [
                matchup_teams[0]['team'].serialized()['name'].decode(),
                matchup_teams[1]['team'].serialized()['name'].decode(),
            ]
        )

2022-11-01 19:21:32.086 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.
2022-11-01 19:21:32.646 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.
2022-11-01 19:21:33.286 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.
2022-11-01 19:21:33.840 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.
2022-11-01 19:21:34.381 - WARNING - query.py - yfpy.query:790 - No game id or season/year provided, defaulting to current fantasy season.


In [320]:
from copy import deepcopy


In [333]:
l_standings

{'Belichicken Tenders': {'wins': 8, 'losses': 0},
 'Tyler to be named later': {'wins': 6, 'losses': 2},
 'Sophkillaaaaaaaaa': {'wins': 5, 'losses': 3},
 "Zach's FUKU Fighters": {'wins': 5, 'losses': 3},
 "Steven's Slam-Dunk Team": {'wins': 5, 'losses': 3},
 "Rachel's Peerless Team": {'wins': 5, 'losses': 3},
 "Ethan Rosenblum's Nifty Team": {'wins': 4, 'losses': 4},
 'McCaf(r)é': {'wins': 3, 'losses': 5},
 'Greg the Leg': {'wins': 3, 'losses': 5},
 'LROZ': {'wins': 2, 'losses': 6},
 'Flour Power': {'wins': 1, 'losses': 7},
 'Intern Season': {'wins': 1, 'losses': 7}}

In [329]:
final_results = pd.DataFrame()
for i in range(10000):
    l_standings_sim = deepcopy(l_standings)
    for matchup in matchups:
        winner = np.random.randint(0,2)
        win_name = matchup[winner]
        lose_name = [i for i in matchup if i != win_name][0]
        l_standings_sim[win_name]['wins'] += 1
        l_standings_sim[lose_name]['losses'] += 1
        
    sim_standings = pd.DataFrame(l_standings_sim).transpose().sample(frac=1)
    sim_standings['rank'] = sim_standings['wins'].rank(method='first', ascending=False)
    sim_standings['playoffs'] = np.where(
        sim_standings['rank'] <= 6,
        1,
        0
    )

    final_results = pd.concat([final_results, sim_standings])

In [330]:
len(final_results)

120000

In [331]:
final_results_names = final_results.reset_index()

In [332]:
final_results_names.groupby('index')['playoffs'].mean()

index
Belichicken Tenders             1.0000
Ethan Rosenblum's Nifty Team    0.4777
Flour Power                     0.0077
Greg the Leg                    0.1886
Intern Season                   0.0052
LROZ                            0.0429
McCaf(r)é                       0.1856
Rachel's Peerless Team          0.7883
Sophkillaaaaaaaaa               0.7761
Steven's Slam-Dunk Team         0.7834
Tyler to be named later         0.9423
Zach's FUKU Fighters            0.8022
Name: playoffs, dtype: float64